# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
import sys
sys.executable

'/miniconda/bin/python'

In [3]:
%env TARGET=10.10.167.66

env: TARGET=10.10.167.66


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

zsh:1: command not found: docker


# Initial Nmap:

In [7]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.167.66


In [8]:
!ping -c 5 $TARGET

PING 10.10.167.66 (10.10.167.66) 56(84) bytes of data.
64 bytes from 10.10.167.66: icmp_seq=1 ttl=64 time=0.760 ms
64 bytes from 10.10.167.66: icmp_seq=2 ttl=64 time=0.450 ms
64 bytes from 10.10.167.66: icmp_seq=3 ttl=64 time=0.471 ms
64 bytes from 10.10.167.66: icmp_seq=4 ttl=64 time=0.428 ms
64 bytes from 10.10.167.66: icmp_seq=5 ttl=64 time=0.475 ms

--- 10.10.167.66 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4101ms
rtt min/avg/max/mdev = 0.428/0.516/0.760/0.122 ms


# Quick Scan:

In [9]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:30 UTC
Initiating ARP Ping Scan at 21:30
Scanning 10.10.167.66 [1 port]
Completed ARP Ping Scan at 21:30, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 21:30
Completed Parallel DNS resolution of 1 host. at 21:30, 0.00s elapsed
Initiating SYN Stealth Scan at 21:30
Scanning ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66) [1000 ports]
Discovered open port 443/tcp on 10.10.167.66
Discovered open port 80/tcp on 10.10.167.66
Completed SYN Stealth Scan at 21:30, 4.75s elapsed (1000 total ports)
Nmap scan report for ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66)
Host is up, received arp-response (0.00033s latency).
Scanned at 2021-10-06 21:30:12 UTC for 5s
Not shown: 997 filtered ports
Reason: 997 no-responses
PORT    STATE  SERVICE REASON
22/tcp  closed ssh     reset ttl 64
80/tcp  open   http    syn-ack ttl 64
443/tcp open   https   syn-ack ttl 64
MAC Address: 02:DC:5F:58:3D:0D (Unk

# All Ports:

In [10]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:30 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 21:30
Scanning 10.10.167.66 [1 port]
Completed ARP Ping Scan at 21:30, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 21:30
Completed Parallel DNS resolution of 1 host. at 21:30, 0.00s elapsed
Initiating SYN Stealth Scan at 21:30
Scanning ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66) [65535 ports]
Discovered open port 80/tcp on 10.10.167.66
Discovered open port 443/tcp on 10.10.167.66
SYN Stealth Scan Timing: About 19.83% done; ETC: 21:32 (0:02:05 remaining)
SYN Stealth Scan Timing: About 48.10% done; ETC: 21:32 (0:01:06 remaining)
Completed SYN Stealth Scan at 21:32, 104.20s elapsed (65535 total ports)
Initiating Service scan at 21:32
Scanning 2 services on ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66)
Completed Service scan at 21:32, 12.03s elapsed (2 services on 1 host)
NSE: Script scanning 10.10.167.

In [11]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [12]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=22,80,443


# No Ping:

In [13]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:32 UTC


Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 21:32
Scanning 10.10.167.66 [1 port]
Completed ARP Ping Scan at 21:32, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 21:32
Completed Parallel DNS resolution of 1 host. at 21:32, 0.00s elapsed
Initiating SYN Stealth Scan at 21:32
Scanning ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66) [1000 ports]
Discovered open port 80/tcp on 10.10.167.66
Discovered open port 443/tcp on 10.10.167.66
Completed SYN Stealth Scan at 21:32, 4.45s elapsed (1000 total ports)
Initiating Service scan at 21:32
Scanning 2 services on ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66)
Completed Service scan at 21:32, 12.03s elapsed (2 services on 1 host)
NSE: Script scanning 10.10.167.66.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 21:32
Completed NSE at 21:32, 0.18s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 21:32
Completed NSE at 21:32, 0.02s elapsed
Nmap scan repo

In [14]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [15]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:32 UTC
NSE: Loaded 123 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 21:32
Completed NSE at 21:32, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 21:32
Completed NSE at 21:32, 0.00s elapsed
Initiating ARP Ping Scan at 21:32
Scanning 10.10.167.66 [1 port]
Completed ARP Ping Scan at 21:32, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 21:32
Completed Parallel DNS resolution of 1 host. at 21:32, 0.00s elapsed
Initiating SYN Stealth Scan at 21:32
Scanning ip-10-10-167-66.eu-west-1.compute.internal (10.10.167.66) [3 ports]
Discovered open port 443/tcp on 10.10.167.66
Discovered open port 80/tcp on 10.10.167.66
Completed SYN Stealth Scan at 21:32, 0.03s elapsed (3 total ports)
NSE: Script scanning 10.10.167.66.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 21:32
Completed NSE at 21:32, 0.30s elapsed
NSE: Starting 

In [16]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:32 UTC
NSE: Loaded 419 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 21:32
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE Timing: About 96.91% done; ETC: 21:33 (0:00:01 remaining)
Completed NSE at 21:33, 40.00s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 21:33
Completed NSE at 21:33, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan.
Initiati

In [17]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-06 21:38 UTC
NSE: Loaded 419 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 21:38
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE Timing: About 95.92% done; ETC: 21:38 (0:00:01 remaining)
Completed NSE at 21:38, 40.05s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 21:38
Completed NSE at 21:38, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan.
Initiati

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt